In [81]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import itertools
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import FeatureUnion
from sklearn.metrics import f1_score, recall_score, precision_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns

In [47]:
df = pd.read_csv("churn_data.csv")
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [48]:
#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [49]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [50]:
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=0)

In [51]:
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [52]:
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [53]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('scaler', StandardScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))

In [54]:
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

In [55]:
rf = Pipeline([
    ('transform', feature_processing),
    ('classifier', RandomForestClassifier(random_state=1, max_depth=5))
])
rf.fit(X_train, y_train)

Pipeline(steps=[('transform',
                 Pipeline(steps=[('feats',
                                  FeatureUnion(transformer_list=[('Geography',
                                                                  Pipeline(steps=[('selector',
                                                                                   FeatureSelector(column='Geography')),
                                                                                  ('ohe',
                                                                                   OHEEncoder(key='Geography'))])),
                                                                 ('Gender',
                                                                  Pipeline(steps=[('selector',
                                                                                   FeatureSelector(column='Gender')),
                                                                                  ('ohe',
                                               

In [73]:
result = pd.DataFrame(columns=['model', 'f1', 'precision', 'recall', 'roc_auc'])

for i in [RandomForestClassifier, CatBoostClassifier, LogisticRegression, XGBClassifier]:
    pipeline = Pipeline([
    ('transform', feature_processing),
    ('classifier', i(random_state=1))
    ])
    pipeline.fit(X_train, y_train)
    pred = pipeline.predict(X_test)
    metrics = {
        'model': i.__name__,
        'f1': f1_score(y_test, pred),
        'precision': precision_score(y_test, rf_pred),
        'recall': recall_score(y_test, rf_pred),
        'roc_auc': roc_auc_score(y_test, rf_pred)
    }
    result = result.append(metrics, ignore_index=True)

In [72]:
result

,model,f1,precision,recall,roc_auc
0,RandomForestClassifier,0.604762,0.860697,0.339882,0.662909
1,CatBoostClassifier,0.625711,0.860697,0.339882,0.662909
2,LogisticRegression,0.334262,0.860697,0.339882,0.662909
3,XGBClassifier,0.608506,0.860697,0.339882,0.662909


#### Я бы назвал лучшей моделью CatBoost, судя по F1-Score, поскольку считаю, что эта метрика наиболее объективно отражает поставленную задачу: мы хотим обнаружить как можно больше уходящих пользователей, чтобы попытаться предотвратить их отток (Presision), но и не хотим слишком часто ошибаться, классифицируя 0 класс как 1 класс, чтобы не тратить деньги на удержание пользователей, которые фактически не уходят от нас (Recall). Обе эти метрики формируют F1-Score

### 3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты.

In [117]:
X_train = X_train[categorical_columns + continuous_columns]
X_test = X_test[categorical_columns + continuous_columns]

In [118]:
model = Pipeline([
    ('transform', feature_processing),
    ('classifier', CatBoostClassifier(silent=True, random_state=1))
    ])
model.fit(X_train, y_train)
pred = model.predict(X_test)
conf_matrix = confusion_matrix(y_test, pred)
print(f'F1-Score = {f1_score(y_test, pred)}')
conf_matrix

F1-Score = 0.6257110352673492


array([[1896,   95],
       [ 234,  275]])

In [119]:
tp = conf_matrix[0, 0]
fp = conf_matrix[0, 1]
fn = conf_matrix[1, 0]
tn = conf_matrix[1, 1]

#### Если считать, что мы не теряем деньги при оттоке клиентов, то выручка будет следующей:
$TP - FP$,
#### поскольку мы не тратим деньги на тех, кого классифицировали как 0, а теряем деньги пытаясь удержать FP

In [120]:
tp - fp

1801

In [153]:
model = CatBoostClassifier(silent=True, random_state=1, learning_rate=1e-3, max_depth=5, 
                                      cat_features=categorical_columns)
model.fit(X_train, y_train)
pred = model.predict(X_test)
conf_matrix = confusion_matrix(y_test, pred)
print(f'F1-Score = {f1_score(y_test, pred)}')
conf_matrix

F1-Score = 0.5605095541401274


array([[1935,   56],
       [ 289,  220]])

In [154]:
tp = conf_matrix[0, 0]
fp = conf_matrix[0, 1]
tp - fp

1879